For any background $B$ and foreground $F$ and matte $\alpha_F$, we can compute a __composition__ $C$

$$
\begin{pmatrix}
    C_r \\
    C_g \\
    C_b \\
\end{pmatrix} = \alpha_F
\begin{pmatrix}
    F_r \\
    F_g \\
    F_b \\
\end{pmatrix}
+ 
(1-\alpha_F)
\begin{pmatrix}
    B_r \\
    B_g \\
    B_b \\
\end{pmatrix}
$$

Given two pair of background $B_0,B_1$ and the corresponding composite $C_0,B_1$, __triangulation matting__ finds foreground color $F$ and mat $\alpha_F$ for all pixel locations. This involves solving the following

$$
\begin{pmatrix}
    C_{0,r} \\
    C_{0,g} \\
    C_{0,b} \\
    C_{1,r} \\
    C_{1,g} \\
    C_{1,b} \\
\end{pmatrix}
= 
\begin{pmatrix}
    1 & 0 & 0 & -B_{0,r} \\
    0 & 1 & 0 & -B_{0,g} \\
    0 & 0 & 1 & -B_{0,b} \\
    1 & 0 & 0 & -B_{1,r} \\
    0 & 1 & 0 & -B_{1,g} \\
    0 & 0 & 1 & -B_{1,b} \\
\end{pmatrix}
\begin{pmatrix}
    \alpha_F F_r \\
    \alpha_F F_g \\
    \alpha_F F_b \\
    \alpha_F
\end{pmatrix}
$$

at each pixel location






In [ ]:
import numpy as np
import scipy.linalg as sp
import cv2 as cv
from matplotlib import pyplot as plt

In [ ]:
#      viscomp.py --matting \
#                 --backA ../test_images/tiny/flowers-backA.jpg \
#                 --backB ../test_images/tiny/flowers-backB.jpg \
#                 --compA ../test_images/tiny/flowers-compA.jpg \
#                 --compB ../test_images/tiny/flowers-compB.jpg \
#                 --alphaOut alpha.tif \
#                 --colOut col.tif

backA = cv.imread("../test_images/tiny/flowers-backA.jpg")
backB = cv.imread("../test_images/tiny/flowers-backB.jpg")
compA = cv.imread("../test_images/tiny/flowers-compA.jpg")
compB = cv.imread("../test_images/tiny/flowers-compB.jpg")

plt.subplot(221); plt.imshow(backA); plt.xticks([]);plt.yticks([])
plt.subplot(222); plt.imshow(backB); plt.xticks([]);plt.yticks([])
plt.subplot(223); plt.imshow(compA); plt.xticks([]);plt.yticks([])
plt.subplot(224); plt.imshow(compB); plt.xticks([]);plt.yticks([])

In [ ]:
# useful for clipping later on with `np.clip`
backA = backA/255.
backB = backB/255.
compA = compA/255.
compB = compB/255.

In [ ]:
# triangulation matting

deltaA = compA-backA
deltaB = compB-backB
colOut = np.zeros(compA.shape)
alphaOut = np.zeros(colOut.shape[:2])

nr,nc,nchannels = compA.shape

for i in range(int(nr/3),int(nr/3)+int(nr/3)):
    for j in range(int(nc/3),int(nc/3)+int(nc/3)):
        b = np.hstack((deltaA[i,j,:], deltaB[i,j,:]))
        b = np.transpose(b)
        A = np.zeros((6,4))
        A[0:3,0:3] = np.eye(3)
        A[3:6,0:3] = np.eye(3)
        A[:,3] = -np.hstack((backA[i,j,:], backB[i,j,:]))
        x = np.dot(np.linalg.pinv(A),b)
        x = np.clip(x,0.,1.)
        
        colOut[i,j,:] = x[:3]
        alphaOut[i,j] = x[3]
        
cv.imwrite('mycol.tif', colOut*255)
cv.imwrite('myalpha.tif', alphaOut*255)
plt.subplot(121); plt.imshow(colOut)
plt.subplot(122); plt.imshow(alphaOut, "gray")

In [ ]:
# 	
# 	 viscomp.py --compositing \
# 	            --alphaIn alpha.tif \
# 	 			--colIn col.tif \
# 	            --backIn ../test_images/tiny/window.jpg \
# 	            --compOut comp.jpg
alphaIn = cv.imread("myalpha.tif") / 255.
colIn = cv.imread("mycol.tif") / 255.
backIn = cv.imread("../test_images/tiny/window.jpg") / 255.

plt.subplot(131); plt.imshow(alphaIn); plt.xticks([]);plt.yticks([])
plt.subplot(132); plt.imshow(colIn); plt.xticks([]);plt.yticks([])
plt.subplot(133); plt.imshow(backIn); plt.xticks([]);plt.yticks([])

In [ ]:
compOut = np.zeros_like(colIn)
nr,nc,nchannels = colIn.shape
for i in range(0,nr):
    for j in range(0,nc):
        alpha = alphaIn[i,j]
        compOut[i,j,:] = alpha*colIn[i,j,:] + (1-alpha)*backIn[i,j,:]
plt.imshow(compOut)

In [ ]:
# vectorized version ...
compOutvec = alphaIn*colIn + (1-alphaIn)*backIn
plt.imshow(compOutvec)